In [3]:
if "descended" not in locals():
    descended = 1
    %cd ".."
    
import utils

import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

import torch

import visualize as vis

/Users/Yke/Desktop/AI/Thesis/ESA/transit-detection-rnn


In [4]:
def plot_results(dic, m, rng=(), split="valid", lbl="", linestyle="-", fill=True, return_c=True, c=None):
    x = np.arange(len(dic["mean"]["metrics"]["train"][()]["acc"]))
    p = plt.plot(x, dic["mean"]["metrics"][split][rng][m], linestyle=linestyle, label=lbl, color=c)
    if fill:
        plt.fill_between(x, dic["mean"]["metrics"][split][rng][m]-dic["std"]["metrics"][split][rng][m],
                        dic["mean"]["metrics"][split][rng][m]+dic["std"]["metrics"][split][rng][m], alpha=0.3, color=p[0].get_color())
    return p[0].get_color() if return_c else None

In [5]:
os.listdir("results/preprocessing")

FileNotFoundError: [Errno 2] No such file or directory: 'results/preprocessing'

## Scaling
Scaling by (white-)noise estimate, standardizing, feeding derivatives

In [ ]:
results = {"mscale0_mode1":'bigru1_w4snr_sim_mscale0_stddize1_mode1_nanmode0.pkl',
           "mscale1_mode1":'bigru1_w4snr_sim_mscale1_stddize1_mode1_nanmode0.pkl',
           "mscale1_mode0":'bigru1_w4snr_sim_mscale1_stddize1_mode0_nanmode0.pkl',
           "mscale0_mode2":'bigru1_w4snr_sim_mscale0_stddize1_mode2_nanmode0.pkl',
           "mscale0_mode3":'bigru1_w4snr_sim_mscale0_stddize1_mode3_nanmode0.pkl'}
for r in results:
    with open("results/preprocessing/"+results[r], "rb") as f:
        results[r] = pickle.load(f)

In [ ]:
dontshow=[]
plt.figure(figsize=(10,4))
for mname in [m for m in results if m not in dontshow]:
    plot_results(results[mname], "acc_seg", split="valid", lbl=mname, fill=1)
plt.legend(fontsize=12, bbox_to_anchor=(1,1))

plt.tight_layout()
plt.show() 

## Data gaps
Simulated gaps, zero-filling, linear interpolation, predicting missing values.

In [ ]:
results = {"nanmode1":'bigru1_w4snr_simgap_mscale1_stddize1_mode1_nanmode1.pkl',
            "nanmode2":'bigru1_w4snr_simgap_mscale1_stddize1_mode1_nanmode2.pkl',
           "nanmode0":'bigru1gen_w4snr_simgap_mscale1_stddize1_mode1_nanmode0.pkl',
           "mscale1_mode1":'bigru1_w4snr_sim_mscale1_stddize1_mode1_nanmode0.pkl'}
for r in results:
    with open("results/preprocessing/"+results[r], "rb") as f:
        results[r] = pickle.load(f)

In [ ]:
dontshow=[]
plt.figure(figsize=(10,4))
for mname in [m for m in results if m not in dontshow]:
    plot_results(results[mname], "tpr", split="valid", lbl=mname, fill=0)
plt.legend(fontsize=12, bbox_to_anchor=(1,1))

plt.tight_layout()
plt.show() 

## Lilith specific
Outlier removal, low-risk detrending (compare vs non-scaled flux segments)